In [63]:
import pandas as pd
from datetime import date
import re

In [2]:
pd.set_option('display.expand_frame_repr', False) # the frame will be huge, don't expand
pd.set_option('display.precision', 4)

In [71]:
demo = pd.read_csv("counties.csv", dtype={'FIPS':float})#  looks like JH data doesn't have leading zeros in FIPS codes
confirmed = pd.read_csv("time_series_covid19_confirmed_US.txt")
deaths = pd.read_csv("time_series_covid19_deaths_US.txt")

In [24]:
demo.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,population,pop_65_plus,health_ins,county,state,FIPS,area,domestic_passengers,intl_passengers,order started
0,"Morgan County, Alabama: Summary level: 050, st...",2.56,580,6009,10431,4473,11938,5507,53742,9.9,...,119089,20464,117677,Morgan County,Alabama,1103.0,579.34,0.0,0.0,04/04/20
1,"Kings County, California: Summary level: 050, ...",3.15,7797,3901,11372,4707,3752,4899,52644,15.6,...,151366,15413,136372,Kings County,California,6031.0,1389.42,0.0,0.0,03/19/20
2,"Monterey County, California: Summary level: 05...",3.31,30494,19232,37518,20736,12420,17109,190707,10.5,...,435594,59491,419413,Monterey County,California,6053.0,3280.60,186000.0,0.0,03/19/20
3,"Nevada County, California: Summary level: 050,...",2.37,596,7285,9199,5133,1562,4717,44505,5.1,...,99696,27746,98472,Nevada County,California,6057.0,957.77,0.0,0.0,03/19/20
4,"Shasta County, California: Summary level: 050,...",2.59,743,6543,17734,7999,3077,8953,69649,9.5,...,180040,37027,178552,Shasta County,California,6089.0,3775.40,0.0,0.0,03/19/20


In [25]:
confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,121,121,128,130,133,133,133,133,135,135
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,6,6,6,6,6,6,6,6,6,6
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,573,620,683,725,788,897,903,923,974,1043
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,43,45,45,50,51,51,51,51,51,51


In [68]:
# get date of at least 10 confirmed cases...
date_10_cases = list()

for x in range(confirmed.shape[0]):
    trans = confirmed.iloc[x].T
    trans = trans.iloc[11:] # just use the date fields
    trans = trans[trans >= 10]
    
    if len(trans) > 0:
        date_10_cases.append(trans.keys()[0])
    else:
        date_10_cases.append(date.today().strftime("%m/%d/%y"))
    

#keys = confirmed.iloc[55].keys()
#values = confirmed.iloc[55].values

In [90]:
confirmed["ten plus cases"] = date_10_cases

In [72]:
deaths.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0.0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,4,4,4,4,5,5,5,5.0,5,5
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,1,1,1,1,1,1,1,1.0,1,1
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,23,24,33,39,42,44,45,45.0,51,56
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,1,1,1,1,1,1,1,1.0,1,1


In [73]:
deaths.shape

(3256, 98)

In [74]:
merged = pd.merge(demo, deaths, how='inner', on="FIPS", 
                  left_index=False, right_index=False)

In [75]:
merged.head()

,Unnamed: 0,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
0,"Morgan County, Alabama: Summary level: 050, st...",2.56,580,6009,10431,4473,11938,5507,53742,9.9,...,0,0,0,0,0,0,0,0.0,0,0
1,"Kings County, California: Summary level: 050, ...",3.15,7797,3901,11372,4707,3752,4899,52644,15.6,...,0,0,0,0,0,1,1,1.0,1,1
2,"Monterey County, California: Summary level: 05...",3.31,30494,19232,37518,20736,12420,17109,190707,10.5,...,2,2,2,3,3,3,3,3.0,3,3
3,"Nevada County, California: Summary level: 050,...",2.37,596,7285,9199,5133,1562,4717,44505,5.1,...,1,1,1,1,1,1,1,1.0,1,1
4,"Shasta County, California: Summary level: 050,...",2.59,743,6543,17734,7999,3077,8953,69649,9.5,...,3,3,3,3,3,3,3,3.0,3,3


In [76]:
merged.describe()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20
count,827.0000,8.2700e+02,8.2700e+02,8.2700e+02,8.2700e+02,8.2700e+02,8.2700e+02,8.2700e+02,827.0000,8.2700e+02,...,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000,827.0000
mean,2.5905,-1.2072e+06,-1.1886e+06,-1.1714e+06,-1.1929e+06,-1.1938e+06,-1.1912e+06,1.6397e+05,9.0312,1.2412e+05,...,14.2370,16.5212,18.5889,20.9166,23.3192,25.1778,26.8972,29.4389,32.4595,37.8875
std,0.2503,3.4773e+07,3.4774e+07,3.4775e+07,3.4774e+07,3.4774e+07,3.4774e+07,2.9310e+05,4.2493,2.0578e+05,...,143.3261,163.4332,184.3480,208.4848,228.2320,247.1663,263.5049,283.8116,304.0160,407.3928
min,1.9000,-1.0000e+09,-1.0000e+09,-1.0000e+09,-1.0000e+09,-1.0000e+09,-1.0000e+09,1.7763e+04,1.3000,2.0411e+04,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,2.4100,5.2650e+02,3.5970e+03,1.0331e+04,4.1055e+03,4.6810e+03,5.0705e+03,4.4176e+04,5.9000,3.6320e+04,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000
50%,2.5500,9.4700e+02,7.0970e+03,1.7522e+04,7.2520e+03,8.2010e+03,9.0380e+03,7.5494e+04,8.4000,6.0368e+04,...,1.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,3.0000,3.0000
75%,2.7100,1.7195e+03,1.8680e+04,4.0398e+04,1.6138e+04,1.6168e+04,1.8663e+04,1.6863e+05,11.4000,1.2666e+05,...,5.0000,6.0000,7.0000,7.0000,8.0000,9.0000,9.0000,10.0000,11.0000,12.5000
max,4.1100,6.6139e+04,6.4235e+05,1.0316e+06,5.8385e+05,4.5555e+05,5.0494e+05,5.0014e+06,29.4000,3.3139e+06,...,4009.0000,4571.0000,5150.0000,5820.0000,6367.0000,6898.0000,7349.0000,7905.0000,8455.0000,11477.0000


In [83]:
merged.columns.values

array(['Unnamed: 0', 'household_size', 'empl_agriculture',
       'empl_professional', 'empl_social', 'empl_services',
       'empl_manufacturing', 'empl_retail', 'employed', 'prc_fam_poverty',
       'avg_income', 'prc_public_transp', 'population', 'pop_65_plus',
       'health_ins', 'county', 'state', 'FIPS', 'area',
       'domestic_passengers', 'intl_passengers', 'order started', 'UID',
       'iso2', 'iso3', 'code3', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population',
       '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20',
       '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20',
       '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20',
       '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20',
       '2/27/20', '2/28/20', '2/29/20', '3/1/2

In [84]:
merged.shape

(827, 119)

In [85]:
merged[["population", "Population"]] 
# ACS population and that used by JH data very close, though not exactly the same
# just use ACS population for consistency
# ultimately drop columns 'UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State',
#        'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', "Unnamed: 0"
# and drop/ignore for model "county", "state", "FIPS"

,population,Population
0,119089,119679
1,151366,152940
2,435594,434061
3,99696,99755
4,180040,180080
...,...,...
822,814901,822083
823,85129,84769
824,948201,945726
825,187365,187885


In [88]:
merged = merged.drop(['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State','Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', "Unnamed: 0"], axis=1)

In [93]:
# add the date of 10+ confirmed deaths as a column
confirmed = confirmed[["FIPS", "ten plus cases"]]

In [95]:
merged = pd.merge(merged, confirmed, how='inner', on="FIPS",
                 left_index=False, right_index=False)

In [97]:
merged.head()

,household_size,empl_agriculture,empl_professional,empl_social,empl_services,empl_manufacturing,empl_retail,employed,prc_fam_poverty,avg_income,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,ten plus cases
0,2.56,580,6009,10431,4473,11938,5507,53742,9.9,45851,...,0,0,0,0,0,0,0.0,0,0,3/28/20
1,3.15,7797,3901,11372,4707,3752,4899,52644,15.6,43727,...,0,0,0,0,1,1,1.0,1,1,4/12/20
2,3.31,30494,19232,37518,20736,12420,17109,190707,10.5,126299,...,2,2,3,3,3,3,3.0,3,3,3/21/20
3,2.37,596,7285,9199,5133,1562,4717,44505,5.1,41447,...,1,1,1,1,1,1,1.0,1,1,3/28/20
4,2.59,743,6543,17734,7999,3077,8953,69649,9.5,68198,...,3,3,3,3,3,3,3.0,3,3,4/3/20


In [99]:
merged.shape

(827, 108)

In [102]:
merged.to_csv("combined_data.csv")